In [1]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000 to get all the rows for our dataframe
# pd.set_option('display.max_colwidth', None)  # or 199


/kaggle/input/tabular-playground-series-sep-2021/train.csv
/kaggle/input/tabular-playground-series-sep-2021/test.csv
/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")

In [3]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")

In [4]:
train_df.head(10)

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.108590,0.004314,-37.566,0.017364,0.289150,-10.251000,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.017700,1
1,1,0.100900,0.299610,11822.000,0.276500,0.459700,-0.837330,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.283590,0
2,2,0.178030,-0.006980,907.270,0.272140,0.459480,0.173270,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.406900,1
3,3,0.152360,0.007259,780.100,0.025179,0.519470,7.491400,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.514860,1
4,4,0.116230,0.502900,-109.150,0.297910,0.344900,-0.409320,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.230490,1
5,5,0.101530,-0.002612,-1118.700,0.116300,0.318860,-0.478390,2372.80,-1808.0,5.818100e+15,...,-44.0820,3.5812,26.55900,-6.47220,44570.00,1.1776,2.997700e+14,5548.3,-0.033159,0
6,6,0.003073,0.359530,20913.000,0.003465,0.268060,-1.225100,1301.20,233180.0,1.771400e+15,...,-39.9220,1.4896,1.29230,33.63400,16201.00,1.2271,2.458900e+16,1012.9,0.647970,0
7,7,0.097340,0.245910,11775.000,0.614860,0.479370,-1.486500,3179.20,123940.0,1.652900e+15,...,-22.9950,1.5719,101.95000,1.51560,1117.50,1.2468,-6.971300e+13,1481.7,0.303170,1
8,8,-0.008948,0.338020,503.810,0.601520,0.261760,0.735380,1614.00,408030.0,7.684400e+14,...,-9.5142,2.2176,-1.23750,7.07130,1030.80,1.2040,5.290300e+15,1968.1,0.145930,0
9,9,0.126230,0.173960,1662.000,0.038081,0.000485,0.059909,296.07,-8035.3,4.526300e+15,...,-1.9684,3.9595,19.16900,-0.24498,838.39,1.1968,2.567000e+17,4914.9,0.519760,0


So, there are some 119 features and and a target variable claim. 
Let's first try to train the model without null values and will then impute the null values with some value. 

In [5]:
Summary = pd.DataFrame(train_df.dtypes, columns=['Dtype'])
Summary["max"] = train_df.max()
Summary["min"] = train_df.min()
Summary["Null"] = train_df.isnull().sum() # to get null values
Summary["First"] = train_df.iloc[0] # to get first value
Summary["Second"] = train_df.iloc[1] # to get second value
Summary


,Dtype,max,min,Null,First,Second
id,int64,9.579180e+05,0.000000e+00,0,0.000000e+00,1.000000e+00
f1,float64,4.151700e-01,-1.499100e-01,15247,1.085900e-01,1.009000e-01
f2,float64,5.189900e-01,-1.904400e-02,15190,4.313900e-03,2.996100e-01
f3,float64,3.954400e+04,-9.421700e+03,15491,-3.756600e+01,1.182200e+04
f4,float64,1.319900e+00,-8.212200e-02,15560,1.736400e-02,2.765000e-01
f5,float64,5.547500e-01,-6.989800e-03,15405,2.891500e-01,4.597000e-01
f6,float64,1.120200e+01,-1.279100e+01,15521,-1.025100e+01,-8.373300e-01
f7,float64,5.426600e+03,-2.248000e+02,15504,1.351200e+02,1.721900e+03
f8,float64,1.913700e+06,-2.984300e+04,15373,1.689000e+05,1.198100e+05
f9,float64,1.042400e+16,-1.153300e+15,15249,3.992400e+14,3.874100e+15


Almost all the rows have null values and the datatypes for all column is float except for claim, that is target variable, in that case it is int.

Let's drop the target variable and id from the training data.

In [6]:
y = train_df["claim"]
train_df = train_df.drop(["id"], axis=1)
train_df = train_df.drop(["claim"], axis=1)

In [7]:
test_df = test_df.drop(["id"], axis=1)

### Adding extra features

In [8]:
train_df["max_value"] = train_df.max(axis = 1)
train_df["min_value"] = train_df.min(axis = 1 )
train_df["mean"] = train_df.mean(axis = 1)
train_df["median"] = train_df.median(axis = 1)
train_df["std"] = train_df.std(axis = 1)
# train_df["unique"] = train_df.nunique(axis = 1)
train_df["null_value"] = train_df.isnull().sum(axis = 1)
print(train_df["median"].tail(10))


957909    5.84505
957910    5.13050
957911    2.64080
957912    3.19960
957913    4.18260
957914    5.82160
957915    4.70020
957916    5.68920
957917    4.59225
957918    6.04800
Name: median, dtype: float64


In [9]:

test_df["max_value"] = test_df.max(axis = 1)
test_df["min_value"] = test_df.min(axis = 1 )
test_df["mean"] = test_df.mean(axis = 1)
test_df["median"] = test_df.median(axis = 1)
test_df["std"] = test_df.std(axis = 1)
# test_df["unique"] = test_df.nunique(axis = 1)
test_df["null_value"] = test_df.isnull().sum(axis = 1)
print(test_df["max_value"].tail(10))

493464    5.889300e+16
493465    2.054000e+16
493466    1.806200e+14
493467    8.641500e+16
493468    1.243100e+16
493469    8.989500e+16
493470    4.691800e+16
493471    7.900900e+16
493472    1.273900e+17
493473    2.459300e+15
Name: max_value, dtype: float64


In [10]:
print(len(train_df))

957919


In [11]:
test_df.head(10)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f115,f116,f117,f118,max_value,min_value,mean,median,std,null_value
0,0.165850,0.487050,1295.00,0.023100,0.31900,0.90188,573.29,3743.7,2.705700e+12,6221.000,...,1.1940,1.980400e+14,2017.100,0.463570,6.046200e+16,-2.218900e+01,1.020407e+15,7.13370,7.739812e+15,1
1,0.129650,0.373480,1763.00,0.728840,0.33247,-1.26310,875.55,554370.0,5.955700e+14,934.430,...,1.1507,4.388000e+16,6638.900,0.281250,4.388000e+16,-1.569100e+01,7.934392e+14,6.58730,5.619785e+15,0
2,0.120190,0.445210,736.26,0.046150,0.29605,0.31665,2659.50,317140.0,3.977800e+14,131.810,...,1.1548,1.718100e+14,5844.000,0.137970,3.977800e+14,-5.953100e+11,8.876224e+12,5.83475,5.348827e+13,1
3,0.054008,0.395960,996.14,0.859340,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,-26.473,...,1.1490,2.138800e+17,6173.300,0.329100,2.138800e+17,-3.432200e+13,3.931683e+15,2.72950,2.746327e+16,0
4,0.079947,-0.006919,10574.00,0.348450,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,5999.400,...,1.2488,2.151300e+17,2250.100,0.337960,2.151300e+17,-3.945700e+08,3.590507e+15,3.73870,2.742801e+16,0
5,0.098700,0.289880,8499.10,0.558120,0.25080,0.11889,3099.90,307200.0,7.736000e+14,5756.700,...,1.3733,1.975900e+14,11405.000,0.843270,3.353000e+16,-3.673300e+10,5.810158e+14,4.53940,4.275462e+15,0
6,NaN,0.495320,2274.40,0.744540,0.43810,3.56510,1534.20,102930.0,8.759700e+14,-2051.000,...,1.3670,4.245900e+16,4488.000,1.616800,4.245900e+16,-7.033500e+04,7.837455e+14,3.56790,5.517502e+15,4
7,0.131640,0.313620,2679.70,0.412410,0.48078,2.66990,1379.10,750760.0,4.893000e+15,-342.500,...,1.2916,1.713500e+16,2386.400,0.060282,1.713500e+16,-3.684200e+11,3.693686e+14,2.94210,2.264836e+15,0
8,0.174400,0.298570,-750.75,0.016088,0.00011,4.17340,667.93,634070.0,8.625400e+14,41552.000,...,1.1299,1.514300e+16,4740.900,0.597500,1.514300e+16,-1.975700e+10,3.212874e+14,6.56440,2.030622e+15,1
9,0.101710,0.123810,594.47,0.034546,0.00075,NaN,2490.20,550700.0,1.283400e+13,127.390,...,NaN,1.146600e+16,-36.192,1.346200,1.146600e+16,-5.189400e+09,2.659247e+14,5.85835,1.649248e+15,3


We can see that the test_df also have 119 features. Target variable is not included in the dataset. 

Let's drop the id column from the test dataset as well and impute the missing values with zero.

In [12]:
# y.nunique.count()

In [13]:
# test_df = test_df.fillna(test_df.mean())


In [14]:
# import matplotlib.pyplot as plt
# corr_mat = train_df.corr()

In [15]:
# cor_target = abs(corr_mat["claim"])
# #Selecting highly correlated features
# relevant_features = cor_target[cor_target<0.0009]
# relevant_features

In [16]:
# feat_to_drop = relevant_features.index.to_list()
# train_df = train_df.drop(feat_to_drop, axis = 1)
# test_df = test_df.drop(feat_to_drop, axis = 1)

In [17]:
# test_df.isnull().sum()
y.value_counts()

0    480404
1    477515
Name: claim, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(train_df, y, stratify = y, random_state = 123, test_size = 0.1)

In [19]:
from sklearn.impute import SimpleImputer

numerical_transformer = SimpleImputer(strategy='constant', fill_value=0)
imputed_train = pd.DataFrame(numerical_transformer.fit_transform(X_train))
imputed_test = pd.DataFrame(numerical_transformer.transform(X_test))
imputed_train.columns = train_df.columns # Columns are not preserved when imputing
imputed_test.columns = train_df.columns

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(imputed_train)
X_test = scaler.transform(imputed_test)

In [21]:
from xgboost import XGBRegressor, XGBClassifier
# import lightgbm

In [22]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

862127
95792
862127
95792


In [ ]:

from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV

model = LogisticRegression()
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold( n_splits=10, n_repeats = 3, random_state = 123 )
grid_search = GridSearchCV( estimator = model, param_grid = grid, n_jobs=-1, cv = cv, scoring = "roc_auc", error_score=0)
grid_result = grid_search.fit(X_train, y_train)

# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [23]:
# lr = LogisticRegression()
lr = LogisticRegression(random_state=123, C = 0.01, penalty = 'l2')
lr.fit(X_train, y_train)


LogisticRegression(C=0.01, random_state=123)

In [24]:
y_pred = lr.predict_proba(X_test)

In [25]:
y_pred = y_pred[:, 1]

In [26]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# this is the score: 0.5254161343242256
# this is the score with logisticR: 0.530162419244593
# score with logistic regression: 0.5410502320900562
#  score with logistic regression with mean: Nevermind
# score with zero imputation: 0.5818869874137387
# score improved without removing extra features that I added 0.5895987572971035
# score improved without removing extra features and adding extra features 0.7981633906567501
# score with unique value feature added did not hep 0.7907597973759513
# score now is 0.798165516631935

0.7981651234333259


In [27]:
# sub = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")
# sub["claim"] = y_val_xgbr
# sub.to_csv("submission.csv", index=False)
# sub.head(10)

In [ ]:
# xgbr= XGBClassifier(
#     n_estimators=3000,
#     learning_rate=0.03,
#     subsample=0.8,
#     colsample_bytree=0.1,
#     max_depth=3,
#     booster='gbtree',
#     reg_lambda=0.0008,
#     reg_alpha=24,
#     random_state=123
# )


xgbr = XGBClassifier(n_estimators=1000, learning_rate=0.05, tree_method='gpu_hist',
                              max_depth=7, n_jobs=-1, min_child_weight=3, gamma=0,
                              reg_alpha=1e-2, reg_lambda=1.5, subsample=0.9, colsample_bytree=0.9, 
                              max_bin=256, objective='binary:logistic', eval_metric='auc', 
                              max_delta_step=0, predictor='gpu_predictor', use_label_encoder=False, random_state = 123)

xgbr.fit(X_train, y_train)

In [ ]:
xgbr.classes_
y_pred_xgb = xgbr.predict_proba(X_test)

In [ ]:
y_pred_xgb = y_pred_xgb[:, 1]

In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred_xgb)
print(auc(fpr, tpr))
# this is the score: 0.776245437662909
#  score with classifier with mean: 0.7680734765097048 (something is fucking wrong)
# score with classifier with mean is: 0.7896067869183678(not a improvement that I wa sexpecting)
# score with classifier with constant imputation: 0.7910729879146615( little bit of improvement, step by step)
# score impored when hyperparams tuned more: 0.7933510777791837
# score without removing the correlated features and removed Standatrdscaler as well
# hope to see the score going up to  bit but don't know what is going to happen 0.8140196515956925 acheived

In [28]:

imputed_test_df = pd.DataFrame(numerical_transformer.transform(test_df))
imputed_test_df.columns = imputed_test_df.columns # Columns are not preserved when imputing

In [29]:
test_df = scaler.transform(imputed_test_df)

In [30]:
test_df

array([[ 1.72557244,  0.96850277, -0.42406785, ...,  1.16798739,
        -0.00795593, -0.44416118],
       [ 0.91532278,  0.21825032, -0.35070447, ...,  0.82242891,
        -0.26116251, -0.93759048],
       [ 0.70358351,  0.6921044 , -0.51165556, ...,  0.34649547,
        -0.9259764 , -0.44416118],
       ...,
       [ 1.16085976,  0.95330879, -0.34535898, ...,  1.50541917,
         0.29592903,  1.52955604],
       [ 1.773695  ,  0.07476627, -0.55577862, ..., -0.03204412,
         1.00754104, -0.93759048],
       [ 0.09676919,  1.06633881, -0.10625394, ..., -1.36494868,
        -0.89384234,  2.02298534]])

In [31]:
y_val = lr.predict_proba(test_df)
y_val = y_val[:, 1]

In [ ]:
# y_pred = lr.predict_proba(test_df)

In [ ]:
y_val_xgbr = xgbr.predict_proba(test_df)

In [ ]:
y_val_xgbr = y_val_xgbr[:, 1]

In [ ]:
y_res = (y_val_xgbr + y_val)/2
y_res

In [32]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")
sub["claim"] = y_val
sub.to_csv("submission.csv", index=False)
sub.head(10)

,id,claim
0,957919,0.413099
1,957920,0.214234
2,957921,0.381972
3,957922,0.249341
4,957923,0.287079
5,957924,0.268551
6,957925,0.779556
7,957926,0.267099
8,957927,0.372764
9,957928,0.668952
